In [1]:
import numpy as np
x = np.linspace(-200,200,5000)
deltax = x[1]-x[0]

%matplotlib tk

from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

In [2]:
def norm(phi):
    norm = np.sum(np.square(np.abs(phi)))*deltax
    return phi/np.sqrt(norm)


def complex_plot(x,y):
    real = np.real(y)
    imag = np.imag(y)
    a,*_ = plt.plot(x,real, ':', label='Re')
    b,*_ = plt.plot(x,imag, ':', label='Im')
    plt.xlim(-2,2)
    p,*_ = plt.plot(x,np.abs(y),label='$\sqrt{P}$')
    return a,b,p
    
def wave_packet(pos=0,mom=0,sigma=0.2):

    return (2 * np.pi * np.square(sigma))**(-1/4) * np.exp(1j*mom*x)*np.exp(-np.square((x-pos)/(2*sigma)),dtype=complex)
#     return  np.exp(1j*mom*x)





def d_dxdx(phi,x=x):
    dphi_dxdx = -2*phi
    dphi_dxdx[:-1] += phi[1:]
    dphi_dxdx[1:] += phi[:-1]
    return dphi_dxdx/np.square(deltax)

def d_dt(phi,h=1,m=1,V=0):
    return 1j*h/2/m * d_dxdx(phi) - 1j*V*phi/h


def rk4(phi, dt, **kwargs):
    k1 = d_dt(phi, **kwargs)
    k2 = d_dt(phi+dt/2*k1, **kwargs)
    k3 = d_dt(phi+dt/2*k2, **kwargs)
    k4 = d_dt(phi+dt*k3, **kwargs)
    return phi + dt/6*(k1+2*k2+2*k3+k4)


def simulate(phi_sim, 
             V=0, 
             steps=100000, 
             dt=0.008, 
             condition=None, 
             save_every=1000):
    simulation_steps = np.zeros((int(steps/save_every), 5000), dtype = complex)
    l = 0
    for i in range(steps):
        phi_sim = rk4(phi_sim,dt,V=V)
#         if condition:
#             phi_sim = condition(phi_sim)
#         phi_sim = norm(phi_sim)
        if i % save_every == 0:
            simulation_steps[l,:] = phi_sim
            l += 1
    return simulation_steps

def box_init(V_0, a_begin, a_end):
        plt.gcf().axes[0].axvspan(a_begin, a_end, alpha=0.2, color='orange', label = r'$V_0$ = {}'.format(V_0))
        plt.xlim(-50,50)
        plt.ylim(-2,2)


def animate(simulation_steps, V_0, a_begin, a_end, init_func = None):
    fig = plt.figure()
    re,im,prob = complex_plot(x,simulation_steps[0])
    plt.xlim(-50,50)
    plt.ylim(-2,2)
    plt.xlabel('Position')
    plt.ylabel(r'$\psi$')
    if init_func:
        init_func()
    
    

    def animation(frame):
        prob.set_data((x, np.abs(simulation_steps[frame])))
        re.set_data((x, np.real(simulation_steps[frame])))
        im.set_data((x, np.imag(simulation_steps[frame])))
        return prob,re,im

    anim = FuncAnimation(fig, animation, frames=int(len(simulation_steps)), init_func = box_init(V_0, a_begin, a_end), interval = 50)
    plt.show()
    plt.legend()
    

    return anim

In [3]:
#parameters

V_0 = [1, 1.5, 2, 2.5, 3]
a_begin = 1.0
a_end = 3.0
width = a_end - a_begin
momentum = 1.5
variance = 0.5
mass = 1
init_pos = -20

In [5]:
#trasmittibility as function of potential

trasmiss = np.zeros((len(V_0)))
for i in range(len(V_0)):
    box_potential = np.where((x>a_begin)&(x<a_end),V_0[i],0)
    sim_box_mom = simulate(wave_packet(pos = init_pos, mom = momentum, sigma = variance),V=box_potential,steps=20000,save_every=100)
    trasmiss[i] = np.sum(np.square(np.abs(sim_box_mom[-1,:])), where=(x>a_end))*deltax

In [4]:
#animating simulation

# box_potential = np.where((x>a_begin)&(x<a_end),V_0,0)
# sim_box_mom = simulate(wave_packet(pos = -20, mom = momentum, sigma = variance),V = box_potential, steps=10000,save_every=100)
# animate(sim_box_mom, 0.01, a_begin, a_end)

In [6]:
#calculating trasmitted and reflected wave

# prob_before_barrier = np.sum(np.square(np.abs(sim_box_mom[-1,:])), where=(x<a_begin))*deltax
# prob_after_barrier = np.sum(np.square(np.abs(sim_box_mom[-1,:])), where=(x>a_end))*deltax
# print('trasmitted : ', prob_after_barrier)
# print('reflected : ', prob_before_barrier)
# print(prob_after_barrier + prob_before_barrier)

In [7]:
E = d_dxdx(wave_packet(pos = init_pos,mom = momentum, sigma= variance))/(2*mass)
energy  = np.sum(np.abs(wave_packet(pos = init_pos,mom = momentum, sigma= variance) * E * np.conjugate(wave_packet(pos = init_pos,mom = momentum, sigma= variance))))*deltax

V = np.linspace(1.01*energy,3.1, 100)

a1 = np.square(V)/(4*energy*(V-energy))
a2 = np.sinh(np.sqrt(2*mass*(V- energy)) * width)


T = 1/(1 +  a1* np.square( a2 ) )
T2 = np.exp(-2 * np.sqrt(2*mass* (V - energy)) * width)

In [14]:
plt.plot(V, T, label = 'theoretical')
plt.scatter(V_0, trasmiss)
plt.legend()